# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



## Imports

In [1]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy, copy
from dataclasses import dataclass
import math
import ipdb

## The *Nim* and *Nimply* classes

In [2]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [3]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Strategies

#### Random strategy

>A completely random move

In [4]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


#### Gabriele's strategy
> Pick always the maximum possible number of the lowest row


In [5]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


#### Optimal strategy

>Fixed rules based on *nim-sum*

In [6]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


#### EA strategy
> A strategy that can adapt its parameters

In [7]:
# Set of available operations on the state
F = ['+', '-', '*', 'div', 'mod', 'and', 'or', 'xor']

POPULATION_SIZE = 50
NUM_ROWS = 5
NUM_GAMES = 40
TOURNAMENT_SIZE = 2
OFFSPRING_SIZE = 50
MUTATION_PROBABILITY = 0.3
NUM_GENERATIONS = 20

In [8]:
def check_result(state: list, nim: Nim, row: int, num: int) -> int:
    '''
        Performs the result of the operation given by the genotype.
        
    '''
    nim = deepcopy(nim)

    result = state[0]

    nim.nimming(Nimply(row, num))

    for i in range(1, len(state), 2):
        operator = state[i]
        n = state[i + 1]

        if operator == "+":
            result = result + nim.rows[n]
        elif operator == "-":
            result = result - nim.rows[n]
        elif operator == "*":
            result = result * nim.rows[n]
        elif operator == "div":
            if nim.rows[n] != 0:
                result = int(result / nim.rows[n])
            else:
                return None
        elif operator == "mod":
            if nim.rows[n] != 0:
                result = result % nim.rows[n]
            else:
                return None
        elif operator == "and":
            result = result & nim.rows[n]
        elif operator == "or":
            result = result | nim.rows[n]
        elif operator == "xor":
            result = result ^ nim.rows[n]

    return result


def fitness(state: tuple[list, int]) -> tuple[int, int]:
    """Number of games won"""

    result = 0
    formula = state[0]
    formula_result = state[1]

    random_cost = 0

    for i in range(NUM_GAMES):
        nim = Nim(NUM_ROWS)
        player = random.choice([0, 1])
        while nim:
            if player == 0:
                ply = optimal(nim)
            elif player == 1:
                already_played = []
                ok = False
                times = 0
                while not ok:
                    times += 1
                    if times > NUM_ROWS**3:
                        break
                    
                    row = random.choice(range(NUM_ROWS))

                    if nim.rows[row] == 0:
                        continue
                    num = random.choice(range(1, nim.rows[row] + 1))

                    if (row, num) in already_played:
                        continue

                    already_played.append((row, num))
                    c_result = check_result(formula, nim, row, num)

                    if c_result != None and c_result % 2 == formula_result:
                        ok = True

                if times > NUM_ROWS**3:
                    ply = pure_random(nim)
                    random_cost += 1
                else:
                    ply = Nimply(row, num)

            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            result += 1
    return result, -random_cost

In [9]:
@dataclass
class Individual:
    genotype: tuple[list, int]
    fitness: int

def select_parent(pop: list[Individual]) -> Individual:
    '''Selects a parent with the maximum fitness from a random pool of individuals'''
    
    pool = [random.choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = deepcopy(ind)
    
    pos = random.randint(0, NUM_ROWS)
    if pos == NUM_ROWS:
        offspring.genotype[1] = 1 - offspring.genotype[1] # Changes the check-module gene
    else:
        ok = False
        while not ok:
            new_value = random.choice(F)
            if pos % 2 == 0:
                if pos == 0:
                    pos = 1
                elif pos == NUM_ROWS:
                    pos = NUM_ROWS - 1
                else:
                    pos += 1
                
            if new_value != offspring.genotype[0][pos]:
                offspring.genotype[0][pos] = new_value # Changes a value in the formula
                ok = True
    
    offspring.fitness = None
    return offspring

def xover(ind1: Individual, ind2: Individual) -> Individual:
    
    cut_point = random.randint(0, NUM_ROWS * 2 - 1)
    
    f_result = random.choice([ind1.genotype[1], ind2.genotype[1]])
    
    offspring = Individual(
        fitness=None, genotype=[ind1.genotype[0][:cut_point] + ind2.genotype[0][cut_point:], f_result] # Mixes the two formulas in one
    )
    
    return offspring

In [10]:
population = [
    Individual(
        genotype=[
            [
                random.choice(F) if i % 2 != 0 else int(i / 2)
                for i in range(NUM_ROWS * 2 - 1)
            ],
            random.choice([0, 1]),
        ],
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)

population

[Individual(genotype=[[0, 'div', 1, 'or', 2, '*', 3, '+', 4], 1], fitness=(15, -79)),
 Individual(genotype=[[0, 'xor', 1, 'and', 2, 'and', 3, '+', 4], 0], fitness=(15, 0)),
 Individual(genotype=[[0, 'and', 1, 'div', 2, 'and', 3, 'or', 4], 0], fitness=(11, -51)),
 Individual(genotype=[[0, '-', 1, 'or', 2, 'and', 3, 'or', 4], 0], fitness=(8, -54)),
 Individual(genotype=[[0, '-', 1, 'div', 2, 'xor', 3, 'div', 4], 1], fitness=(16, -95)),
 Individual(genotype=[[0, 'and', 1, '-', 2, 'div', 3, '-', 4], 1], fitness=(8, -63)),
 Individual(genotype=[[0, 'xor', 1, 'xor', 2, '-', 3, '*', 4], 1], fitness=(8, -59)),
 Individual(genotype=[[0, '*', 1, 'xor', 2, 'div', 3, 'mod', 4], 0], fitness=(9, -80)),
 Individual(genotype=[[0, 'mod', 1, 'and', 2, 'and', 3, '+', 4], 0], fitness=(13, -53)),
 Individual(genotype=[[0, 'or', 1, '*', 2, 'mod', 3, '*', 4], 1], fitness=(8, -137)),
 Individual(genotype=[[0, 'mod', 1, '-', 2, 'or', 3, 'or', 4], 1], fitness=(8, -82)),
 Individual(genotype=[[0, 'mod', 1, '+', 

In [11]:
for generation in range(NUM_GENERATIONS):
        offspring = list()
        
        for counter in range(OFFSPRING_SIZE):
            if random.random() < MUTATION_PROBABILITY:
                p = select_parent(population)
                o = mutate(p)
            else:
                p1 = select_parent(population)
                p2 = select_parent(population)
                o = xover(p1, p2)
            offspring.append(o)
    
        for i in offspring:
            i.fitness = fitness(i.genotype)
        
        population.extend(offspring)
        population.sort(key=lambda i: i.fitness, reverse=True)
        population = population[:POPULATION_SIZE]
        print(population[0])


Individual(genotype=[[0, '*', 1, '+', 2, 'xor', 3, '-', 4], 1], fitness=(23, -23))
Individual(genotype=[[0, '*', 1, '+', 2, 'xor', 3, '-', 4], 1], fitness=(23, -23))
Individual(genotype=[[0, '*', 1, '+', 2, 'xor', 3, '-', 4], 1], fitness=(23, -23))
Individual(genotype=[[0, '+', 1, '+', 2, 'xor', 3, '-', 4], 1], fitness=(24, -18))
Individual(genotype=[[0, 'xor', 1, 'or', 2, '+', 3, '-', 4], 1], fitness=(29, -11))
Individual(genotype=[[0, 'xor', 1, 'or', 2, '+', 3, '-', 4], 1], fitness=(29, -11))
Individual(genotype=[[0, 'xor', 1, 'or', 2, '+', 3, '-', 4], 1], fitness=(29, -11))
Individual(genotype=[[0, 'xor', 1, 'or', 2, '+', 3, '-', 4], 1], fitness=(29, -11))
Individual(genotype=[[0, 'xor', 1, 'or', 2, '+', 3, '-', 4], 1], fitness=(29, -11))
Individual(genotype=[[0, 'xor', 1, 'or', 2, '+', 3, '-', 4], 1], fitness=(29, -11))
Individual(genotype=[[0, '-', 1, '+', 2, '+', 3, '-', 4], 1], fitness=(30, -10))
Individual(genotype=[[0, '-', 1, '+', 2, '+', 3, '-', 4], 1], fitness=(30, -10))
In

In [12]:
def adaptive(nim: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""

    formula = population[0].genotype[0]
    formula_result = population[0].genotype[1]
    ok = False
    times = 0
    already_played = []
    while not ok:
        times += 1
        if times > NUM_ROWS**3:
            break
        row = random.choice(range(NUM_ROWS))
        
        if nim.rows[row] == 0:
            continue
        num = random.choice(range(1, nim.rows[row] + 1))
        if (row, num) in already_played:
            continue

        already_played.append((row, num))
        c_result = check_result(formula, nim, row, num)

        if c_result != None and c_result % 2 == formula_result:
            ok = True

    if times > NUM_ROWS**3:
        ply = pure_random(nim)
    else:
        ply = Nimply(row, num)

    return ply

### Player 0

### Player 1

## Oversimplified match

In [44]:
logging.getLogger().setLevel(logging.INFO)



strategy = (optimal, adaptive)

nim = Nim(NUM_ROWS)
logging.info(f"init : {nim}")
player = random.choice([0,1])
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=6)
INFO:root:status: <1 3 5 1 9>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 3 4 1 9>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=4)
INFO:root:status: <1 3 4 1 5>
INFO:root:ply: player 1 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 4 1 5>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=4)
INFO:root:status: <0 3 0 1 5>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=2)
INFO:root:status: <0 3 0 1 3>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=2)
INFO:root:status: <0 1 0 1 3>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=2)
INFO:root:status: <0 1 0 1 1>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=1)
INFO:root:status: <0 1 0 0 1>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=1)
INFO:root:status: <0 1 0 0 0>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=1)
INFO:root:status: <0 0